In [ ]:
import folium
import polars as pl
import geopandas as gpd

import fryer.data

pl.Config.set_tbl_rows(100)

# Road Accidents from 1979 to Latest Published Year

### UK Gov - Dept for Transport

[Source](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data)

[Guidance](https://www.gov.uk/guidance/road-accident-and-safety-statistics-guidance)

**Load a dictionary of road accident data**

keys are the dataset names below and values are the corresponding polars dataframes

- collision
- caualties
- vehicles

Casualties and vehicles can be joined via a zero or many to one relationship with the collision dataframe using the `accident_index` column

Additionally, `vehicle_reference` in the casualties and vehicles table will relate a casualty to a particular vehicle involved in the matching accident

[Local Authority borders](https://geoportal.statistics.gov.uk/search?q=BDY_LAD%202024&sort=Title%7Ctitle%7Casc)

In [ ]:
dfs = fryer.data.uk_gov_dept_for_transport_road_accidents.read()
gdf = fryer.data.ons_local_authority_district_boundaries.read()
collision, casualty, vehicle = dfs.values()

for name, df in dfs.items():
    print(name, "\nrows:", "{:,}".format(df.shape[0]), "\ncolumns:", df.shape[1], "\n")

In [ ]:
collision["local_authority_ons_district"].null_count()

In [ ]:
collision = collision.filter(pl.col("local_authority_ons_district").is_not_null())

collisions_count = collision.group_by("local_authority_ons_district").agg(
    pl.col("accident_index").count().alias("accidents")
)

gdf2 = gdf.merge(
    collisions_count.to_pandas(),
    left_on="LAD24CD",
    right_on="local_authority_ons_district",
    how="left",
)

gdf2["geometry"] = gpd.GeoSeries(gdf2["geometry"]).simplify(tolerance=0.001)


m = folium.Map()
cp = folium.Choropleth(
    geo_data=gdf2,
    name="LA_Test",
    data=gdf2,
    columns=["LAD24CD", "accidents"],
    key_on="feature.properties.LAD24CD",
    fill_color="Reds",
    bins=8,
    fill_opacity=0.7,
    line_opacity=0.5,
    smooth_factor=0,
    # use_jenks=True,
    overlay=True,
    highlight=True,
).add_to(m)

folium.GeoJsonTooltip(["LAD24CD", "accidents"]).add_to(cp.geojson)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
raw_df = pl.read_csv(
    "/media/linkside/hdd/scratch/eel/uk_gov_dept_for_transport_road_accidents_raw/collision-1979-latest-published-year.csv",
    ignore_errors=True,
)

In [ ]:
# px.scatter(
#     collision.filter(
#         pl.col('accident_year') == 2023,
#         pl.col('accident_severity') != 'Slight',
#     ),
#     x="longitude",
#     y="latitude",
#     color="accident_severity",
#     hover_data=["number_of_vehicles", "number_of_casualties"],
#     height=800,
# )

In [ ]:
# px.histogram(vehicle, x="age_of_driver", title='Age of Driver Distribution')

In [ ]:
# year = 2023
# local_authority = "Croydon"

# df = collision.filter(
#     pl.col('accident_year') == year,
#     pl.col('local_authority') == local_authority,
#     pl.col('accident_severity') != 'Slight'
# )

# # instantiate a feature group for the incidents in the dataframe
# road_accidents = folium.map.FeatureGroup()

# # loop through the 100 crimes and add each to the incidents feature group
# for lat, lng, in zip(df["latitude"], df["longitude"]):
#     road_accidents.add_child(
#         folium.features.CircleMarker(
#             [lat, lng],
#             radius=5, # define how big you want the circle markers to be
#             color='yellow',
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6
#         )
#     )

# # add pop-up text to each marker on the map
# latitudes = list(df["latitude"])
# longitudes = list(df["longitude"])
# labels = list(df["severity"])

# collision_map = folium.Map(location=[latitudes[0], longitudes[0]], zoom_start=12) #, tiles='Stamen Terrain')

# for lat, lng, label in zip(latitudes, longitudes, labels):
#     folium.Marker([lat, lng], popup=label).add_to(collision_map)

# # add incidents to map
# collision_map.add_child(road_accidents)

In [ ]:
df.head()